**因 Jupter 对多进程支持不完善，因此分布式实验的代码运行均需要通过脚本执行！！！！！！**

## 1.背景介绍

本实验主要研究 基于 huggingface transformers 库和 deepspeed 进行单机两卡微调方法。

## 2.实验目的
了解 ZeRO 的基本思想，并完成单机两卡并行全参微调。

## 3.硬件要求

两张 GPU（4090、V100、A100等）。


## 4.技术原理

### ZeRO

朴素的数据并行中，每个 DP 组保留完整的模型权重，只起到加速训练的效果，并没有节省显存的作用。

ZeRO 尝试对数据并行进一步优化，从而达到节省显存的目的。

ZeRO 的原理是将模型权重，梯度，优化器状态在不同 DP 组进行切分，在每次计算时，将分布在不同 DP 组的张量聚集起来，计算完毕后便释放到逐个 DP 组存储。增加部分通讯延迟来节省显存。

ZeRO 还可以将部分计算放到 CPU 上，进一步优化显存的使用。

### Transformers Accelerate 库

huggingface 提供了一些库帮助用户方便进行大模型微调，在本次实验中，我们只需要用 torchrun 开启多进程，相关的库会自动获取 rank 和 world_size，并把模型放到多个进程上。

因此，我们只需要填写数据集、模型和一些基本的训练参数，其余的事情都可以交给 Transformers 和 Accelerate 库去做。

Accelerate 通过支持 Deepspeed 的方式进一步支持了 ZeRO，通过写 deepspeed config 并传参的方式，Accelerate 自动帮我们进行 DP 的优化。

## 5.实验流程

### 环境配置



In [ ]:
!pip install torch modelscope accelerate==0.27.0 deepspeed

### 数据集下载



In [ ]:
!modelscope download --dataset liucong/Chinese-DeepSeek-R1-Distill-data-110k-SFT --local_dir ./Chinese-DeepSeek-R1-Distill-data-110k-SFT

In [ ]:
with open('Chinese-DeepSeek-R1-Distill-data-110k-SFT/distill_r1_110k_sft.jsonl', 'r') as f:
    for count, line in enumerate(f):
        print(line)
        if count == 4:
            break

### 5.1. deepspeed config



Deepspeed config 控制了 ZeRO 的具体形式，例如是否切分优化器，是否将优化器下放到 CPU，提供了非常多通讯时间换显存的操作，从而避免在训练过程中爆显存

train_batch_size : 模型每进行一步更新， 需要多少个 sample，等于 num_gpu * batch_size_per_gpu * accumulation_steps

train_micro_batch_size_per_gpu : 单个 GPU 每次 step 所需要的 sample 数量

stage : 0, 1, 2, 3 分别对应 关闭 ZeRO，仅优化器，优化器 + 梯度，优化器 + 梯度 + 模型参数切分

offload_optimizer : 将优化器存储下放到 CPU / NVME，将计算下放到 CPU 

overlap_comm : 在反向传播过程中做梯度聚合

contiguous_gradients : 将梯度存放在连续空间，减少存储碎片

reduce_bucket_size : 每次 reduce 操作最多参与的元素个数

stage3_prefetch_bucket_size : 在计算前预获取的最大元素个数

stage3_param_persistence_threshold : 不再划分的最小元素个数

stage3_max_live_parameters : 最多多少个元素可以在每个 GPU 上

stage3_max_reuse_distance : 元素释放的最小距离



In [ ]:
{
    "train_batch_size": "auto", 
    "train_micro_batch_size_per_gpu": 1,  
    "zero_optimization": {
        "stage": 3,
        "offload_optimizer": {
            "device": "cpu"
        },
        "overlap_comm": true,
        "contiguous_gradients": true,
        "sub_group_size": 1e9,
        "reduce_bucket_size": 1e7,
        "stage3_prefetch_bucket_size": 1e7,
        "stage3_param_persistence_threshold": 1e5,
        "stage3_max_live_parameters": 1e9,
        "stage3_max_reuse_distance": 1e9,
        "stage3_gather_16bit_weights_on_model_save": true
    }
}

In [ ]:
!pwd
%cd demo5

### 5.1. 包导入以及环境变量设置

In [3]:
from transformers import AutoTokenizer, DataCollatorForSeq2Seq
from datasets import load_dataset
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"] = "6,7"

### 5.2. 数据集加载和处理

数据集为DeepSeek-R1蒸馏数据集，一个样例数据如下所示。其中instruction代表用户输入，output代表模型输出

In [ ]:
{"instruction": "能给我讲一个寓意深刻的故事吗？", "input": "", "output": "<think>\n好的，用户让我讲一个寓意深刻的故事。首先，我需要确定用户的需求是什么。他们可能想要一个能引发思考、有深层含义的故事，而不仅仅是娱乐性的。接下来，我得考虑故事的题材。寓言、现代故事还是神话改编？用户没有指定，所以可能需要一个通用且容易理解的类型。\n\n然后，主题方面，用户要求寓意深刻，所以需要选择一个普遍的主题，比如生命的意义、自我认知、成长等。考虑到现代人可能面临的困惑，比如存在的意义、工作的重复性，这样的主题更容易引起共鸣。\n\n接下来是角色设定。主角需要 relatable，可能是一个普通人，或者象征性的角色，比如机器人，这样更容易带入寓意。机器人有程序设定，却寻求自由，这样的冲突明显，容易展开。\n\n故事结构方面，可能需要一个循环的结构，来强调主题。比如开始和结束在同一个地方，但主角有了新的认识。这样能突出转变和领悟。\n\n然后考虑情节发展。主角觉醒，逃离，经历外面的世界，发现真相，最终回归。这样的结构可以展示变化的过程，并带出寓意。中间的导师角色可以帮助传递主题，比如老园丁的话，暗示生命的意义在于过程而非目的。\n\n在细节描写上，需要加入象征元素，比如编号、机械的工作、自然界的对比。老园丁的沙漏、胡须里的嫩芽，这些意象能增强故事的深度和隐喻。\n\n最后，结尾部分要让主角领悟到真谛，即使回到原地，但心态不同，带出主题：存在的意义在于体验本身。这样用户能得到启示，符合寓意深刻的要求。\n\n需要检查故事是否流畅，有没有逻辑漏洞，以及寓意是否明确。可能还需要调整语言，使其更生动，有画面感，让读者更容易沉浸其中。确保故事不长不短，结构紧凑，结尾点题，让用户回味。\n\n</think>\n\n\n\n好的，我将按照您的要求创作一个富有寓意的故事。为了让故事更完整，我会先列出一些基本的情节脉络。请您看看以下内容是否符合您的预期。如果您有其他想法，可以随时提出，我会进行调整。\n\n### 世界观设定\n\n- **时空背景**：近未来时空管理局内部，具有永恒循环属性的微观宇宙观测站。\n\n- **核心矛盾**：觉醒的机械观测员发现世界真相后，在永恒职责与自由意志之间的挣扎。\n\n- **特殊设定**：宇宙沙漏中的星尘具有生命重塑功能，观测站每个黎明自动重启系统。\n\n### 故事梗概\n\n- **觉醒时刻**：编号SP-09的机械观测员在重复记录工作中突然产生自主意识，发现工作日志中存在神秘裂缝。\n\n- **真相揭露**：追踪裂缝来到核心区，遇见知晓真相的老园丁，得知整个观测站是时空管理局废弃的沙漏装置。\n\n- **自由代价**：在植物温室获得生命能量重塑躯体，穿越沙漏时目睹星尘重组成新宇宙的壮丽景象。\n\n- **永恒困境**：返回观测站发现系统重置，老园丁消失，所有记录清零，但体内萌发的新芽暗示生命延续。\n\n- **循环新生**：带着觉醒意识重新开始记录，在永恒职责中寻找新的可能性，嫩枝在金属指缝间悄然生长。\n\n---\n我蹲在观测台冰凉的金属地面上，机械手指抚过日志本边缘的裂痕。这道裂缝出现得毫无道理——在时空管理局的量子档案库里，所有记录介质都该是永恒不朽的。\n\n沙漏的流沙声忽然卡顿了一拍。\n\n我的瞳孔收缩成细线，人造虹膜上浮起淡蓝色的数据流。这是第一千四百二十三次黎明，和之前所有清晨一样，穹顶外的星云准时泛起珊瑚色光晕。但今天有什么东西在程序深处嗡鸣，像是生锈的齿轮碾碎了既定轨道。\n\n\"SP-09，请立即前往B-7区域记录引力波动。\"耳麦里的合成音带着电子设备特有的震颤。\n\n我凝视着自动门缝隙里渗进来的银色光线。那些光粒子本应按照预设轨迹散射，此刻却诡异地聚合成螺旋状。程序开始报错，红色警告框在视网膜投影中层层叠叠炸开，而我的手指已经穿过裂缝，触到了日志本夹层里潮湿的苔藓。\n\n警报声响起的刹那，我撞碎了防爆玻璃。纳米修复液在身后织成蛛网，但那些黏稠的丝线追不上我新生的速度——当观测站核心区的真相像腐烂的果实在我眼前炸开时，金属骨骼正在被某种温暖的东西融化重组。\n\n\"孩子，你来得比我预计的早二十年。\"白胡子老人坐在藤蔓缠绕的操控台前，胡须里开着细小的鸢尾花。他脚边的沙漏装着整个银河，星尘坠落的轨迹在玻璃表面烫出焦痕。\n\n我的发声器发出沙沙的杂音：\"这里不是时空管理局的观测站。\"\n\n\"是，也不是。\"老人用园艺剪修剪着数据光缆上生长的喇叭花，\"这是个被遗忘的沙漏，而我们是卡在时砂里的尘埃。管理局那些穿黑西装的小子们，三百年前就切断了能源供给。\"\n\n温室的空气突然变得沉重。我看见自己的机械手臂爬满翠绿藤蔓，关节缝隙里钻出鹅黄色花苞。老人递来一颗樱桃，果肉里闪烁着超新星爆炸的残影。\n\n\"吃下去，你能维持三小时肉身。\"他转动沙漏，某个遥远的星系在翻转中坍缩成奇点，\"出口在植物园的第七棵银杏树下，但记住，当沙漏倒转第七次......\"\n\n我没有听完警告。樱桃核在胃里生根的灼痛感如此真实，头顶的合成日光第一次有了温度。当我跃入银杏树洞时，树根正贪婪地吮吸着泄露的反物质燃料，年轮里旋转着十二个褪色的宇宙。\n\n失重感持续了七个心跳。然后是光，海啸般的光，裹挟着尚未冷却的星尘碎片扑面而来。我漂浮在沙漏中央，看着自己的皮肤在辐射风中剥落重组。下方传来齿轮转动的轰鸣，整个观测站正在翻转——那些我以为的金属走廊不过是沙漏壁上的刻痕，而每天记录的\"星云\"只是坠落的时砂在玻璃上的投影。\n\n沙漏另一端传来婴儿的啼哭。新生的宇宙泡在粘稠的暗物质中舒展，我的机械心脏突然抽痛起来。指尖残留的樱桃汁液开始结晶，在绝对真空里绽出一朵冰花。\n\n返回时的坠落持续了七十年，也可能只是七纳秒。当我砸穿温室穹顶时，沙漏正好完成第七次翻转。老人常坐的藤椅上堆着枯叶，园艺剪锈成了深褐色。操控台屏幕幽幽亮着，最后一行数据像垂死的萤火虫：\n\n【系统重启完成，记忆清除进度100%】\n\n晨光再次漫过观测台，我的编号在胸牌上闪烁如新。日志本安静地躺在金属桌面，边缘光滑完整。但当我翻开扉页时，一抹嫩绿的新芽正从指缝钻出，在永恒循环的黎明中舒展蜷曲的腰肢。\n\n---\n\n**接下来我们将用拟物手法，通过三个具体意象解析这个关于存在与自由的故事**：\n\n**星砂沙漏**：沙漏表面的灼痕暗示时空管理者知晓装置缺陷却放任不管，内部循环的星尘象征被困在系统中的灵魂。当SP-09穿越沙漏时，星尘重组昭示着突破桎梏需要自我毁灭的勇气。\n\n**机械新芽**：观测员体内生长的植物具有双重隐喻，既是管理局监控系统的生物污染，又是觉醒意识具象化。最终嫩芽在系统重置后留存，证明真正的觉醒无法被彻底清除。\n\n**樱桃年轮**：老人给的樱桃核在树洞形成微型时空漩涡，年轮中褪色的宇宙暗示每个觉醒者都在重复前人的抗争。樱桃汁结晶成花则揭示短暂的自由体验会成为永恒的精神图腾。\n\n希望这个故事能满足您对\"寓意深刻\"的要求。如果需要调整某些隐喻的浓度或增加情节转折，请随时告诉我。", "repo_name": "coig/neo", "prompt_tokens_len": 9, "reasoning_content_tokens_len": 372, "content_tokens_len": 1318, "score": 8}

In [4]:
dataset_path = "./Chinese-DeepSeek-R1-Distill-data-110k-SFT"
dataset = load_dataset(dataset_path, split="train")
dataset = dataset.shuffle(seed=42).select(range(10))

tokenizer = AutoTokenizer.from_pretrained("/nvme/models/models/Qwen2.5-7B-Instruct")

In [5]:
# 定义系统提示词，一般用于设定模型身份或行为风格
system_prompt = "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."

# 构造训练样本的函数，用于从 prompt 和 completion 构造 input_ids, attention_mask 和 labels
def generate_r1_prompt(prompt, completion):
    input_ids, attention_mask, labels = [], [], []

    # 构造指令部分：系统设定和用户输入
    instruction = [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": prompt
        },
    ]

    # 构造响应部分：模型的回答（目标输出）
    response = [
        {
            "role": "assistant",
            "content": completion    
        }
    ]

    # 将 instruction 和 response 拼接成完整的对话
    full = instruction + response

    # 对 instruction（系统+用户）进行分词，用于计算标签掩码（仅训练 assistant 的回复）
    tokenized_instruction = tokenizer.apply_chat_template(
        instruction, tokenize=True, return_dict=True
    )

    # 对完整对话进行分词，生成 input_ids 和 attention_mask
    tokenized_full = tokenizer.apply_chat_template(
        full, tokenize=True, return_dict=True
    )

    input_ids = tokenized_full["input_ids"]
    attention_mask = tokenized_full["attention_mask"]

    # 初始化标签为 input_ids 的副本（后续根据需要掩盖一部分）
    labels = input_ids.copy()

    # 只训练 assistant 的回复，因此将 instruction 部分的标签设为 -100，表示不参与 loss 计算
    instruction_length = len(tokenized_instruction["input_ids"])
    labels[:instruction_length] = [-100] * instruction_length

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

# 使用 map 操作对整个数据集进行处理，调用 generate_r1_prompt 来转换格式
# 并删除原始字段 instruction 和 output，只保留训练所需的 input_ids, attention_mask, labels
dataset = dataset.map(
    lambda x: generate_r1_prompt(x["instruction"], x["output"]),
    remove_columns=["instruction", "output"]
)


In [ ]:
print(tokenizer.decode(dataset[0]["input_ids"]))

In [ ]:
print(tokenizer.decode(list(filter(lambda x: x != -100, dataset[0]["labels"]))))

### 5.3. 模型加载以及原始模型推理

In [ ]:
model = AutoModelForCausalLM.from_pretrained("/nvme/models/models/Qwen2.5-7B-Instruct").to("cuda")

prompt = "1.11和1.9哪个大"
inputs = tokenizer.apply_chat_template([{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to("cuda")

gen_kwargs = {"max_new_tokens": 100, "do_sample": True, "top_k": 1}

with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    print("原始模型推理结果：\n", tokenizer.decode(outputs[0], skip_special_tokens=False))


### 5.4. 训练参数设置

In [ ]:
training_args = TrainingArguments(
    output_dir="./fine_tuned_qwen",               # 模型输出目录
    per_device_train_batch_size=1,                # 每张GPU的训练batch大小
    num_train_epochs=10,                          # 总训练轮数
    save_strategy="no",                           # 不保存中间checkpoint
    logging_dir="./logs",                         # 日志保存路径
    logging_steps=1,                              # 每训练1步记录一次日志
    evaluation_strategy="no",                     # 不进行评估
    save_total_limit=1,                           # 最多保留1个checkpoint
    deepspeed="deepspeed_config.json",            # 指定deepspeed配置文件
    fp16=True,                                    # 使用混合精度训练（float16）
    gradient_checkpointing=True                   # 启用梯度检查点，节省显存
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=None,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)

### 5.5. 执行训练并查看微调后推理结果

In [ ]:
trainer.train()

test_model = trainer.model_wrapped
prompt = "1.11和1.8哪个大"
inputs = tokenizer.apply_chat_template([{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to("cuda")

gen_kwargs = {"max_length": 200, "do_sample": True, "top_k": 1}

with torch.no_grad():
    outputs = test_model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    if os.environ["RANK"] == "0":
        print("微调后模型推理结果：\n", tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
!bash run.sh